In [16]:
import praw
import re
from datetime import datetime, timedelta
import csv
import pandas as pd

In [17]:
# Reddit API credentials
reddit = praw.Reddit(
    client_id="JVgubQ5eSbuKnFa5PCL1kA",
    client_secret="jO5Bt6c5BGr3q2_rBS7VWsZ9p4JJ8Q",
    user_agent="Ok_Culture7133"
)

In [20]:
# Function to identify state mentioned in the post
def find_state(text):
    states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 
              'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
              'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
              'NewHampshire', 'NewJersey', 'NewMexico', 'NewYork', 'NorthCarolina', 'NorthDakota', 'Ohio', 
              'Oklahoma', 'Oregon', 'Pennsylvania', 'RhodeIsland', 'SouthCarolina', 'SouthDakota', 'Tennessee', 
              'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'WestVirginia', 'Wisconsin', 'Wyoming']
    for state in states:
        if state.lower() in text.lower():
            return state
    return 'national'

# List of subreddits including state-specific and politics-related ones
subreddits = [
    'politics', 'uspolitics', 'republic', 'democrat', 'liberal', 'askpolitics', 'politicaldiscussion','conservative','freethought', 'changemyview', 'moderatepolitics','neutralpolitics', 'trump', 'harris', 'kamalaharris', 'donaldtrump', 'joebiden', 'biden'
]

# Add all 50 states and their possible politics subreddits
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 
          'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
          'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
          'NewHampshire', 'NewJersey', 'NewMexico', 'NewYork', 'NorthCarolina', 'NorthDakota', 'Ohio', 
          'Oklahoma', 'Oregon', 'Pennsylvania', 'RhodeIsland', 'SouthCarolina', 'SouthDakota', 'Tennessee', 
          'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'WestVirginia', 'Wisconsin', 'Wyoming']

# Add state-specific subreddits and their potential "politics" counterparts
for state in states:
    subreddits.append(state.lower())  # State subreddit
    subreddits.append(f'{state.lower()}politics')  # State-politics subreddit

# Define the time window for the last 6 months
six_months_ago = datetime.utcnow() - timedelta(days=180)

# List to store all posts
post_data = []

# Define a broad set of election-related keywords
election_keywords = ['election', 'vote', 'voting', 'voter', 'ballot', 'poll', 'campaign', 'candidate', 'electorate', 'debate', 
                     'primary', 'caucus', 'congress', 'senate', 'representative', 'governor', 'mayor', 'president']

# Political affiliation keywords to check
political_keywords = ['Trump', 'Harris', 'Republican', 'Democrat', 'Donald', 'Kamala', 'Red', 'Blue']

# Fetch posts from the subreddits
for subreddit_name in subreddits:
    try:
        subreddit = reddit.subreddit(subreddit_name)

        # Fetch posts from the last 6 months
        for submission in subreddit.new(limit=None):
            post_date = datetime.utcfromtimestamp(submission.created_utc)
            if post_date < six_months_ago:
                continue
            
            # Extract post details
            title = submission.title
            body = submission.selftext
            votes = submission.score
            comments = submission.num_comments
            post_url = submission.url
            political_affiliation = None
            relevant_post = False

            # Identify if the post is related to elections by checking for election-related keywords
            for keyword in election_keywords:
                if re.search(rf'\b{keyword}\b', title, re.IGNORECASE) or re.search(rf'\b{keyword}\b', body, re.IGNORECASE):
                    relevant_post = True
                    break

            # If the post isn't relevant, skip it
            if not relevant_post:
                continue

            # Check for political affiliation based on specific political keywords
            for keyword in political_keywords:
                if re.search(rf'\b{keyword}\b', title, re.IGNORECASE) or re.search(rf'\b{keyword}\b', body, re.IGNORECASE):
                    political_affiliation = keyword
                    break
            
            # Check for state mentions
            state_mentioned = find_state(title + " " + body)

            # Append the post data to the list
            post_data.append({
                'Subreddit': subreddit_name,
                'Title': title,
                'Body': body,
                'Votes': votes,
                'Comments': comments,
                'Date': post_date.strftime('%Y-%m-%d %H:%M:%S'),
                'State': state_mentioned,
                'Political Affiliation': political_affiliation,
                'URL': post_url
            })
    except Exception as e:
        print(f"Error processing subreddit {subreddit_name}: {e}")


C:\Users\amrut\AppData\Local\Temp\ipykernel_7224\1781421323.py:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  six_months_ago = datetime.utcnow() - timedelta(days=180)
C:\Users\amrut\AppData\Local\Temp\ipykernel_7224\1781421323.py:52: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  post_date = datetime.utcfromtimestamp(submission.created_utc)


Error processing subreddit harris: received 404 HTTP response
Error processing subreddit donaldtrump: received 404 HTTP response
Error processing subreddit connecticutpolitics: received 403 HTTP response
Error processing subreddit floridapolitics: received 403 HTTP response
Error processing subreddit georgiapolitics: received 404 HTTP response
Error processing subreddit michiganpolitics: received 404 HTTP response
Error processing subreddit northcarolinapolitics: received 403 HTTP response
Error processing subreddit rhodeislandpolitics: received 403 HTTP response
Error processing subreddit vermontpolitics: received 403 HTTP response


In [26]:
import unicodedata

# Function to normalize text (convert special characters to plain equivalents)
def normalize_text(text):
    if text:
        return unicodedata.normalize('NFKD', text)
    return text

# Write data to CSV file with proper encoding
csv_file = 'reddit_political_posts.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Subreddit', 'Title', 'Body', 'Votes', 'Comments', 'Date', 'State', 'Political Affiliation', 'URL'])
    writer.writeheader()
    for post in post_data:
        # Normalize text fields to remove any problematic special characters
        post['Title'] = normalize_text(post['Title'])
        post['Body'] = normalize_text(post['Body'])
        post['Political Affiliation'] = normalize_text(post['Political Affiliation'])
        writer.writerow(post)

print(f"Data successfully saved to {csv_file}")


Data successfully saved to reddit_political_posts.csv


In [27]:
df = pd.read_csv('reddit_political_posts.csv')

In [28]:
df.shape

(10269, 9)

In [24]:
df.head

<bound method NDFrame.head of       Subreddit                                              Title  \
0      politics  Tarrant County officials clear up ballot contr...   
1      politics  Bob and Kristina Lange, Republican farmers who...   
2      politics  Politics website labels Nevada's US Senate rac...   
3      politics  LA Times, Calif.'s largest newspaper, won't en...   
4      politics  Senate battle goes from bad to worse for Democ...   
...         ...                                                ...   
10264   wyoming  Wyoming political group makes expensive, murky...   
10265   wyoming  Wyoming’s Title IX lawsuit highlights governor...   
10266   wyoming  UW President Seidel closes university's DEI Of...   
10267   wyoming                          Old political commercial    
10268   wyoming  Wyoming voids 28% of its voter registrations i...   

                                                    Body  Votes  Comments  \
0                                                   